# Introduction and setup

## Magic Methods

> Note: the magic methods should not have any statement after it (including any semi-colons, parenthesis, etc...).

| Magic Command                 | Applies To  | Description                                                          |
| :---------------------------- | :---------: | :------------------------------------------------------------------: |
| %jars <REGEX>                 | Single Line | Specifies the jars via local path to be loaded into the environment. |
| %%compile <FILENAME>          | Whole Cell  | Saves and compiles the entirety of the cell contents as a java file. |
| %maven <NAME:PACKAGE:VERSION> | Single Line | Imports the specified dependency using Gradle like specification.    |
| %compiled                     | Single Line | Returns the path to the latest compiled class file.                  |
| %cclasses                     | Single Line | Returns an ArrayList<String> of all of the compiled class files.     |
| %jvm <java|java7|android>     | Single Line | Retrieves the dynamically specified jvm from the sdkman repository.  |
| %writeOut <FILEPATH>          | Whole Cell  | Writes the entirety of the cell contents to the specified file path. |
| %system                       | Single Line | Runs the specified command via shell and prints the response.        |


## Imports

In [ ]:
// Importing the local CryptoGuard Jar
List <String> addedJars = %jars *.jar

// Maven Imports
%maven junit:junit:4.12
%maven commons-io:commons-io:2.7
%maven com.binarytweed:quarantining-test-runner:0.0.3

// Gradle imports
import static org.junit.Assert.assertFalse;
import static org.junit.Assert.assertTrue;
import static org.junit.Assert.assertEquals;
import static org.junit.Assert.assertNull;
import static org.junit.Assert.assertNotNull;
import static util.Utils.makeArg;
import frontEnd.Interface.outputRouting.ExceptionHandler;
import frontEnd.Interface.ArgumentsCheck;
import frontEnd.MessagingSystem.routing.Listing;
import frontEnd.MessagingSystem.routing.EnvironmentInformation;
import frontEnd.MessagingSystem.routing.structure.Default.Report;
import frontEnd.MessagingSystem.routing.structure.Scarf.AnalyzerReport;
import frontEnd.MessagingSystem.routing.structure.Scarf.BugInstance;
import frontEnd.Interface.EntryPoint;
import rule.engine.EngineType;
import frontEnd.argsIdentifier;
import java.io.File;
import java.nio.charset.StandardCharsets;
import java.nio.file.Files;
import java.nio.file.Paths;
import java.util.ArrayList;
import java.util.Arrays;
import java.util.List;
import org.junit.After;
import org.junit.Before;
import org.junit.Test;
import org.junit.runner.RunWith;
import soot.G;
import util.Utils;
import org.apache.commons.io.FileUtils;
import java.util.Scanner;
import com.binarytweed.test.Quarantine;
import com.binarytweed.test.QuarantiningRunner;
import java.io.File;

## Helpful Utility Functions

In [ ]:
// region String Display
public static void print(String file) {
	try {
		Scanner reader = new Scanner(new File(file));

		while (reader.hasNextLine()) 
			System.out.println(reader.nextLine());
	}
	catch (FileNotFoundException e) {
	}
}

* You can find the source code for the [AnalyzerReport here](https://github.com/CryptoGuardOSS/cryptoguard/blob/master/src/main/java/frontEnd/MessagingSystem/routing/structure/Scarf/AnalyzerReport.java).

## Testing Method Setup

In [ ]:
//This is a curated testing method to simply scan the .class file path passed in.
//__DO NOT CHANGE UNLESS YOU KNOW WHAT YOU ARE DOING__
@RunWith(QuarantiningRunner.class)
@Quarantine({"com.binarytweed", "*"})
public class main_TestableFile_VerySimple {
	public static AnalyzerReport scanfile(String source) {
		G.reset();
		soot.G.v().reset();

		String vm = %jvm java
		String fileOut = source + ".xml";
		AnalyzerReport output = null;

			String args =
				makeArg(argsIdentifier.FORMAT, EngineType.CLASSFILES)
				+ makeArg(argsIdentifier.FORMATOUT, Listing.ScarfXML)
				+ makeArg(argsIdentifier.SOURCE, source)
				+ makeArg(argsIdentifier.NOEXIT)
				+ makeArg(argsIdentifier.STREAM)
				+ makeArg(argsIdentifier.PRETTY)
				+ makeArg(argsIdentifier.JAVA, vm)
				+ makeArg(argsIdentifier.OUT, fileOut);

			try {
				EntryPoint.main(args.split(" "));
				output = AnalyzerReport.deserialize(new File(fileOut));
				//print(fileOut);
			} catch (Exception e) {
				e.printStackTrace();
				assertNull(e);
			}
	return output;
	}
}

# Sample Structure

## Sample Class being compiled

In [ ]:
%%compile sample.java

//Taken From https://github.com/CryptoAPI-Bench/CryptoAPI-Bench/blob/master/src/main/java/org/cryptoapi/bench/brokenhash/BrokenHashABICase10.java

import java.security.MessageDigest;
import java.security.NoSuchAlgorithmException;

public class sample {
    public static void main(String [] args) throws NoSuchAlgorithmException {
        String str = "abcdef";
        String crypto = "MD5";
        method2(str,crypto);
    }
    public static void method2(String s, String cryptoHash) throws NoSuchAlgorithmException {
        String str2 = s;
        String hashAlgo = cryptoHash;
        method1(str2,hashAlgo);
    }
    public static void method1(String str, String crypto) throws NoSuchAlgorithmException {
        MessageDigest md = MessageDigest.getInstance(crypto);
        md.update(str.getBytes());
        System.out.println(md.digest());
    }
}

## Java Class Test

In [ ]:
String source = %compiled
AnalyzerReport result = main_TestableFile_VerySimple.scanfile(source);
System.out.println(result);